<a href="https://colab.research.google.com/github/aryaganu10-cpu/GenAi/blob/main/Required_Task_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# ================================
# Risk Divergence Analysis Script
# ================================

!pip install pypdf
import os
import re
from pypdf import PdfReader
from openai import OpenAI
from google.colab import userdata

# --------------------------
# 1. Setup OpenAI Client
# --------------------------

client = OpenAI(api_key=userdata.get("SECRET_NAME"))

# --------------------------
# 2. PDF Extraction Function
# --------------------------

def extract_text_from_pdf(path):
    reader = PdfReader(path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

# --------------------------
# 3. Extract Risk Factors Section
# --------------------------

def extract_risk_section(text):
    risk_start = re.search(r"Item\s+1A\.\s+Risk\s+Factors", text, re.IGNORECASE)
    risk_end = re.search(r"Item\s+1B\.", text, re.IGNORECASE)

    if risk_start and risk_end:
        return text[risk_start.start():risk_end.start()]
    else:
        return text  # fallback if regex fails

# --------------------------
# 4. Load 10-K Files
# --------------------------

block_text = extract_text_from_pdf("Block 10K 2023.pdf")
paypal_text = extract_text_from_pdf("PayPal 10K 2023.pdf")

block_risk = extract_risk_section(block_text)
paypal_risk = extract_risk_section(paypal_text)

# Truncate to manageable length
block_risk = block_risk[:20000]
paypal_risk = paypal_risk[:20000]

# --------------------------
# 5. Prompt Variations
# --------------------------

prompts = {
    "baseline": """
Compare the Risk Factors sections of Block and PayPal.
Identify operational, legal, and strategic risk differences.
Focus on Bitcoin, AI, and ecosystem expansion vs. efficiency strategy.
""",
    "forensic": """
Act as a forensic financial analyst.
Detect subtle wording shifts, newly introduced risk disclosures,
and/or potential undisclosed operational vulnerabilities.
Focus on AI, Bitcoin, TBD, and ecosystem expansion.
""",
    "contrarian": """
Assume Block's AI & Bitcoin expansion hides operational fragility.
Argue whether their risk disclosures understate legal, regulatory,
or liquidity exposure compared to PayPal.
"""
}

# --------------------------
# 6. Run Analysis Prompts
# --------------------------

analysis_results = {}

for key, prompt in prompts.items():
    response = client.chat.completions.create(
        model="gpt-5",
        messages=[
            {"role": "system", "content": "You are a senior SEC risk analyst."},
            {"role": "user", "content": prompt +
             "\n\nBLOCK RISK FACTORS:\n" + block_risk +
             "\n\nPAYPAL RISK FACTORS:\n" + paypal_risk}
        ],
        temperature=0.3
    )
    analysis_results[key] = response.choices[0].message.content

# --------------------------
# 7. LLM as Judge
# --------------------------

judge_prompt = """
You are an expert financial risk professor.
Evaluate which analysis best identifies undisclosed
operational or legal risks in Block compared to PayPal.
Rate each from 1-10 and explain why.
"""

judge_response = client.chat.completions.create(
    model="gpt-5",
    messages=[
        {"role": "system", "content": "You are an impartial evaluator."},
        {"role": "user", "content": judge_prompt +
         "\n\nBASELINE:\n" + analysis_results["baseline"] +
         "\n\nFORENSIC:\n" + analysis_results["forensic"] +
         "\n\nCONTRARIAN:\n" + analysis_results["contrarian"]}
    ],
    temperature=0
)

judge_output = judge_response.choices[0].message.content

print("===== JUDGE EVALUATION =====")
print(judge_output)

# --------------------------
# 8. Generate Final Executive Summary
# --------------------------

synopsis_prompt = """
Write a 200-word Executive Summary answering:
Is Block's AI & Bitcoin focus creating undisclosed
operational risks compared to PayPal's conservative strategy?
Use the judge's evaluation and strongest findings.
"""

synopsis_response = client.chat.completions.create(
    model="gpt-5",
    messages=[
        {"role": "system", "content": "You are a strategic consultant writing to a board of directors."},
        {"role": "user", "content": synopsis_prompt +
         "\n\nJUDGE OUTPUT:\n" + judge_output}
    ],
    temperature=0.4
)

executive_summary = synopsis_response.choices[0].message.content

print("\n===== EXECUTIVE SUMMARY =====")
print(executive_summary)

# --------------------------
# 9. Optional: Save Results
# --------------------------

with open("risk_divergence_results.txt", "w", encoding="utf-8") as f:
    f.write("JUDGE EVALUATION:\n")
    f.write(judge_output)
    f.write("\n\nEXECUTIVE SUMMARY:\n")
    f.write(executive_summary)

print("\nResults saved to risk_divergence_results.txt")

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: AIzaSyAE***************************vqks. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}